In [1]:
import os
import re
import glob
import ctypes
import rasterio
import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio import mask
from rasterio.plot import show
from rasterio.transform import from_origin
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from osgeo import ogr, gdal, osr
from rasterio.merge import merge
from geopy.distance import geodesic
from rasterio.windows import Window
from shapely.geometry import mapping
from rasterio.enums import Resampling
from shapely.geometry import Point, MultiPoint

In [2]:
def latlon_to_pixel(lat, lon, geotransform):
    
    """
    Convert latitude and longitude to pixel coordinates in a GeoTIFF file.

    Parameters:
    - lat: Latitude
    - lon: Longitude
    - geotransform: GeoTransform information from the GeoTIFF file

    Returns:
    - x_pixel: Pixel x-coordinate
    - y_pixel: Pixel y-coordinate
    """

    # Extracting GeoTransform parameters
    x_geo, pixel_width, _, y_geo, _, pixel_height = geotransform

    # Calculating pixel coordinates
    x_pixel = int((lon - x_geo) / pixel_width)
    y_pixel = int((lat - y_geo) / pixel_height)

    return x_pixel, y_pixel

In [3]:
def get_lat_lon_from_pixel(geo_transform, col, row):
    
    """
    Convert pixel coordinates in a GeoTIFF file to latitude and longitude.

    Parameters:
    - column
    - row
    - geotransform: GeoTransform information from the GeoTIFF file

    Returns:
    - latitude
    - longitude
    """
    
    lon = geo_transform[0] + col * geo_transform[1] + row * geo_transform[2]
    lat = geo_transform[3] + col * geo_transform[4] + row * geo_transform[5]
    return lat, lon

In [4]:
def get_geotransform_from_file(tif_path):
    """
    Get GeoTransform information from a GeoTIFF file.

    Parameters:
    - tif_path: Path to the GeoTIFF file

    Returns:
    - geotransform: GeoTransform information as a tuple
    """
    dataset = gdal.Open(tif_path)
    geotransform = dataset.GetGeoTransform()
    dataset = None  # Close the dataset
    return geotransform

In [5]:
def merge_tiles(year):
    # Input directory containing the TIFF files
    input_directory = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/GFC/GFC_Tiles/20{year}_GFC_tiles'

    # Get a list of all TIFF files in the directory
    input_files = glob.glob(os.path.join(input_directory, '*.tif'))
    
    # print(f"Input directory: {input_directory}")
    # print(f"Input files: {input_files}")

    # Check if there are any TIFF files
    if not input_files:
        print("No TIFF files found in the specified directory.")
        return

    # Read all input files
    src_files_to_mosaic = [rasterio.open(file) for file in input_files]

    # Merge files
    mosaic, out_trans = merge(src_files_to_mosaic, resampling=Resampling.nearest)

    # Output file
    output_file = "/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS/merged_output.tif"

    # Update metadata of the output file
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({"driver": "GTiff", "height": mosaic.shape[1], "width": mosaic.shape[2], "transform": out_trans, "dtype": src_files_to_mosaic[0].dtypes[0]})

    # Write the mosaic to the output file
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

In [6]:
def get_day_from_filename(file_path):
    # Use regular expression to extract the day from the filename
    match = re.search(r'doy2016(\d{3})', file_path)
    if match:
        return int(match.group(1))
    else:
        return 0  # Return 0 if day information is not found

In [8]:
# Create ordered list of APPEARS tifs
pattern = f'MOD13Q1.061__250m_16_days_EVI_doy20{year}*.tif'
tifs = glob.glob(f'{APPEARS_dir}/{pattern}')
        
# Sort the file paths based on the day of the year
appears_tifs = sorted(tifs, key=get_day_from_filename)

day = 1
for snapshot in appears_tifs:
    # Creates appears.tif with same crs and spatial resolution as prodes data
    gdal_command_4 = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -of GTiff {snapshot} appears.tif'
    subprocess.run(gdal_command_4, shell=True)
    
    height, width = prodes_data.shape
    
    prodes_geotransform = get_geotransform_from_file(prodes_path)
    appears_geotransform = get_geotransform_from_file(snapshot)

NameError: name 'year' is not defined

In [7]:
PRODES = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/PRODES/prodes_amazonia_raster_2000_2022_v20231109/prodes_amazonia_raster_2000_2022_v20231109.tif'
ecoregions_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/Ecoregions/Brazilian_Amazon_Ecoregions'
APPEARS_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI'
merged_output_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS'
GFC_directory = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/GFC/GFC_Tiles'

# List of ecoregion .shp filenames
ecoregions = [filename for filename in os.listdir(ecoregions_dir) if filename.endswith('.shp')]

# Path to shared object
lib = ctypes.CDLL('./libsub_arrays.so')

for ecoregion in ecoregions:
    # Path to ecoregion shapefile
    shp_path = os.path.join(ecoregions_dir, ecoregion)
    gdf = gpd.read_file(shp_path)
    
    # Analyze 2008-2023
    for year in range(16,23):
        # !rm *.tif
        # !rm *.xml
        '''
        # Create prodes raster 
        gdal_command_1 = f'gdal_calc.py -A {PRODES} --outfile=out.tif --calc="A=={year}" --NoDataValue=255'
        gdal_command_2 = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -cutline {shp_path} -crop_to_cutline -dstnodata 255 -of GTiff out.tif prodes_{ecoregion}_20{year}.tif'
        subprocess.run(gdal_command_1, shell=True)
        # print('hello')
        subprocess.run(gdal_command_2, shell=True)
        os.remove('out.tif')
        '''
        prodes_path = f'prodes_{ecoregion}_20{year}.tif'
        with rasterio.open(prodes_path) as prodes:
            prodes_data = prodes.read(1)
        
            
        prodes_geotransform = get_geotransform_from_file(prodes_path)
        lat, lon = get_lat_lon_from_pixel(prodes_geotransform, 0, 0)
        height, width = prodes_data.shape
        diff = np.zeros((height, width), dtype=np.float32)
        # prodes_data_bytes = prodes_data.tobytes()
        # prodes_data = np.frombuffer(prodes_data_bytes, dtype=np.float32).reshape((height, width))
        print(f'diff mean: {np.mean(diff)}')
        '''
        # Create GFC raster
        merge_tiles(year)
        gdal_command_3 = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -cutline {shp_path} -crop_to_cutline -dstnodata 255 -of GTiff merged_output.tif gfc_{ecoregion}_20{year}.tif'
        subprocess.run(gdal_command_3, shell=True)
        os.remove('merged_output.tif')
        '''
        gfc_path = f'gfc_{ecoregion}_20{year}.tif'
        with rasterio.open(gfc_path) as gfc:
            gfc_data = gfc.read(1)
            
        # gfc_data_bytes = gfc.tobytes()
        # gfc_data = np.frombuffer(gfc_data_bytes, dtype=np.float32).reshape((height, width))

        # Create ordered list of APPEARS tifs
        pattern = f'MOD13Q1.061__250m_16_days_EVI_doy20{year}*.tif'
        tifs = glob.glob(f'{APPEARS_dir}/{pattern}')
        
        # Sort the file paths based on the day of the year
        appears_tifs = sorted(tifs, key=get_day_from_filename)
        
        day = 1
        
        !rm appears.tif
        
        # Analysis for each .tif of each year
        for snapshot in appears_tifs:
            print(day)
            # Creates appears.tif with same crs and spatial resolution as prodes data
            gdal_command_4 = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -of GTiff {snapshot} appears.tif'
            subprocess.run(gdal_command_4, shell=True)
            
            # Could also use gdal to open raster instead
            with rasterio.open('appears.tif') as apps:
                appears_data = apps.read(1)
                
            appears_geotransform = get_geotransform_from_file(snapshot)
            os.remove('appears.tif')
            # Convert coordinate to upper left pixel
            # Extracting GeoTransform parameters
            x_geo, pixel_width, _, y_geo, _, pixel_height = appears_geotransform
            # Calculating pixel coordinates
            x = int((lon - x_geo) / pixel_width)
            y = int((lat - y_geo) / pixel_height)
            # x, y = latlon_to_pixel(lat, lon, appears_geotransform)
            
            current = appears_data[x:x+height, y:y+width]
            
            if day == 1.0:   
                prev = appears_data[x:x+height, y:y+width]       
            else: # Perform c++ calculations
                # Convert numpy arrays to ctypes pointers
                print(prev.dtype, current.dtype, diff.dtype)
                prev_ptr = prev.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
                current_ptr = current.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
                diff_ptr = diff.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
                lib.sub_arrays(prev_ptr, current_ptr, diff_ptr, height, width)
                prev = current
                '''
                for i in range(height):
                    for j in range(width):
                        if diff[i,j] != 0:
                            print(diff[i,j])
                '''
            
            if day > 1:
                mask = (prodes_data == 1)
                selected_values = diff[mask]
                selected_values *= .0001
                print(f'Average drop in deforestated pixels for day {day}: {np.mean(selected_values)}')

                mask = (prodes_data == 0)
                selected_values = diff[mask]
                selected_values *= .0001
                print(f'Average drop in non-deforestated pixels for day {day}: {np.mean(selected_values)}')
            
            day += 16
            
            !rm appears.tif
            !rm appears.tif.aux.xml
            
        # Breaks year loop
        break
        
    # Breaks ecoregion loop
    break
    
# !rm *.tif

diff mean: 0.0
rm: appears.tif: No such file or directory
1
Creating output file that is 113701P x 81536L.
Processing /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016001_aid0001.tif [1/1] : 0Using internal nodata values (e.g. -3000) for image /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016001_aid0001.tif.
Copying nodata values from source /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016001_aid0001.tif to destination appears.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
rm: appears.tif: No such file or directory
rm: appears.tif.aux.xml: No such file or directory
17
Creating output file that is 113701P x 81536L.
Processing /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016017_aid0001.tif [1/1] :

Processing /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016145_aid0001.tif [1/1] : 0Using internal nodata values (e.g. -3000) for image /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016145_aid0001.tif.
Copying nodata values from source /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016145_aid0001.tif to destination appears.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
int16 int16 float32
Average drop in deforestated pixels for day 145: -0.18120752274990082
Average drop in non-deforestated pixels for day 145: -0.1597633808851242
rm: appears.tif: No such file or directory
rm: appears.tif.aux.xml: No such file or directory
161
Creating output file that is 113701P x 81536L.
Processing /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD

Processing /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016289_aid0001.tif [1/1] : 0Using internal nodata values (e.g. -3000) for image /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016289_aid0001.tif.
Copying nodata values from source /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD13Q1.061__250m_16_days_EVI_doy2016289_aid0001.tif to destination appears.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
int16 int16 float32
Average drop in deforestated pixels for day 289: -0.19121505320072174
Average drop in non-deforestated pixels for day 289: -0.1693636029958725
rm: appears.tif: No such file or directory
rm: appears.tif.aux.xml: No such file or directory
305
Creating output file that is 113701P x 81536L.
Processing /Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI/MOD

In [8]:
            '''
            deforested = []
            non_def = []
            
            lon_array = prodes_geotransform[0] + np.arange(width) * prodes_geotransform[1] + np.arange(height)[:, np.newaxis] * prodes_geotransform[2]
            lat_array = prodes_geotransform[3] + np.arange(width) * prodes_geotransform[4] + np.arange(height)[:, np.newaxis] * prodes_geotransform[5]

            points = MultiPoint([Point(lon, lat) for lon, lat in zip(lon_array.ravel(), lat_array.ravel())])
            within_ecoregion = gdf.geometry.contains(points)

            x_array = np.array(((lon_array - appears_geotransform[0]) / appears_geotransform[1]).astype(int))
            y_array = np.array(((lat_array - appears_geotransform[3]) / appears_geotransform[5]).astype(int))

            for i in tqdm(range(height), desc="Processing"):
                mask = within_ecoregion[i, :]
                val = appears_data[x_array[i, mask], y_array[i, mask]] - prev_apps[i, mask]
                prodes_max[i, mask] = np.maximum(val, prodes_max[i, mask])

                if day == 353 and np.any(prodes_data[i, :] == 1):
                    deforested.append(prodes_max[i, mask])
                elif day == 353:
                    non_def.append(prodes_max[i, mask])

                prev_apps[i, mask] = appears_data[x_array[i, mask], y_array[i, mask]]

            '''
            '''
            for i in tqdm(range(height), desc="Processing"):
                for j in range(width):
                    # lat, lon = get_lat_lon_from_pixel(prodes_geotransform, i, j)
                    
                    lon = prodes_geotransform[0] + j * prodes_geotransform[1] + i * prodes_geotransform[2]
                    lat = prodes_geotransform[3] + j * prodes_geotransform[4] + i * prodes_geotransform[5]
                    point = Point(lon, lat)
                    
                    # Check if coordinates within .shp file
                    if gdf.geometry.contains(point).any():
                        # x, y = latlon_to_pixel(lat, lon, appears_geotransform)
                        
                        # Extracting GeoTransform parameters
                        x_geo, pixel_width, _, y_geo, _, pixel_height = appears_geotransform

                        # Calculating pixel coordinates
                        x = int((lon - x_geo) / pixel_width)
                        y = int((lat - y_geo) / pixel_height)
                        
                        if day >= 17:
                            val = appears_data[x, y] - prev_apps[i, j]
                            if val < prodes_max[i,j]:
                                prodes_max[i,j] = val
                        if day == 353 and prodes_data == 1:
                            deforested.append(prodes_max[i,j])
                        elif day == 353:
                            non_def.append(prodes_max[i,j])
                            
                        prev_apps[i,j] = appears_data[x,y]
    
            '''

'\nfor i in tqdm(range(height), desc="Processing"):\n    for j in range(width):\n        # lat, lon = get_lat_lon_from_pixel(prodes_geotransform, i, j)\n        \n        lon = prodes_geotransform[0] + j * prodes_geotransform[1] + i * prodes_geotransform[2]\n        lat = prodes_geotransform[3] + j * prodes_geotransform[4] + i * prodes_geotransform[5]\n        point = Point(lon, lat)\n        \n        # Check if coordinates within .shp file\n        if gdf.geometry.contains(point).any():\n            # x, y = latlon_to_pixel(lat, lon, appears_geotransform)\n            \n            # Extracting GeoTransform parameters\n            x_geo, pixel_width, _, y_geo, _, pixel_height = appears_geotransform\n\n            # Calculating pixel coordinates\n            x = int((lon - x_geo) / pixel_width)\n            y = int((lat - y_geo) / pixel_height)\n            \n            if day >= 17:\n                val = appears_data[x, y] - prev_apps[i, j]\n                if val < prodes_max[i,j]